In [ ]:
import os
from ase.db import connect
from urllib.request import urlretrieve
from mpcontribs.client import load_client

In [ ]:
project = 'dtu'
db = 'https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db'
client = load_client('your-api-key-here')

Retrieve and update project info

In [ ]:
client.projects.get_entry(pk=project, _fields=['_all']).result()

In [ ]:
client.projects.update_entry(pk=project, project={'urls': {'DTU|DB': db}}).result()

Create contributions

In [ ]:
dbfile = db.rsplit('/', 1)[-1]
if not os.path.exists(dbfile):
    urlretrieve(db, dbfile) 

con = connect(dbfile)
nr_mpids = con.count(selection='mpid')
print(nr_mpids)

In [ ]:
for idx, row in enumerate(con.select('mpid')):
    identifier = f'mp-{row.mpid}'
    print(identifier)
    contrib = {'project': project, 'identifier': identifier, 'data': {
        'ΔE|KS': { # kohn-sham band gap
            'indirect': f'{row.gllbsc_ind_gap - row.gllbsc_disc} eV',
            'direct': f'{row.gllbsc_dir_gap - row.gllbsc_disc} eV'            
        },
        'ΔE|QP': { # quasi particle band gap
            'indirect': f'{row.gllbsc_ind_gap} eV',
            'direct': f'{row.gllbsc_dir_gap} eV'           
        },
        'C': f'{row.gllbsc_disc} eV' # derivative discontinuity
    }}

    ntries = 0
    while ntries < 3:
        try:
            for d in client.contributions.get_entries(
                project=project, identifier=identifier, _fields=['id']
            ).result()['data']:
                client.contributions.delete_entry(pk=d['id']).result()
                #print(d['id'], 'deleted')

            cid = client.contributions.create_entry(contribution=contrib).result()['id']
            #print(client.contributions.get_entry(pk=cid, _fields=['_all']).result())
            #print(cid, 'created')
            break
        except Exception as ex:
            ntries += 1
            print(ntries, ex)
            print('waiting', ntries*30, 'seconds...')
            sleep(ntries*30)
    else:
        print('I give up.')
        break